In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.cuda as cuda

In [2]:
import os
import torch

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids

In [3]:
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        
        super(RNNModel, self).__init__()
        
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            # self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
            
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        #if tie_weights:
        #    if nhid != ninp:
        #        raise ValueError('When using the tied flag, nhid must be equal to emsize')
        #    self.decoder.weight = self.encoder.weight

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()),
                    Variable(weight.new(self.nlayers, bsz, self.nhid).zero_()))
        else:
            return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())

In [4]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [5]:
corpus = Corpus('./data/ohenry')

In [6]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    if cuda.is_available():
        data = data.cuda()
    return data

In [7]:
bs_train = 20
bs_valid = 10
bptt = 35
clip = 0.25
log_interval = 200

In [8]:
train_data = batchify(corpus.train, bs_train)
val_data = batchify(corpus.valid, bs_valid)
test_data = batchify(corpus.test, bs_valid)

In [9]:
ntokens = len(corpus.dictionary)
model = RNNModel('LSTM', ntokens, 200, 200, 2, 0.2, True)
if cuda.is_available():
    model.cuda()

criterion = nn.CrossEntropyLoss()

In [10]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [11]:
def get_batch(source, i, evaluation=False):
    seq_len = min(bptt, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    return data, target

In [12]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0
    #start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(bs_train)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.data
        """
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss[0] / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
        """

In [13]:

def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(bs_valid)
    for i in range(0, data_source.size(0) - 1, bptt):
        data, targets = get_batch(data_source, i, evaluation=True)
        output, hidden = model(data, hidden)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).data
        hidden = repackage_hidden(hidden)
    return total_loss[0] / len(data_source)


In [14]:
# Loop over epochs.
lr = 20
best_val_loss = None

In [15]:
for epoch in range(1, 10):
    # epoch_start_time = time.time()
    train()
    val_loss = evaluate(val_data)
    print(val_loss)
    
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "./4.model.pth")
            
    else:
        print('Annealing')
        lr /= 2.0
        
    """
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
            'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                       val_loss, math.exp(val_loss)))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open(args.save, 'wb') as f:
            torch.save(model, f)
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    """

6.639132971819092
6.218705038783936
5.9514008351057175
5.864150663080196
5.789859584167396
5.742576512260728
5.722437797134993
5.728060392687351
Annealing
5.665329722100588


In [16]:
num_words = 200
temperature = 1

In [20]:
model = RNNModel('LSTM', ntokens, 200, 200, 2, 0.2, True)
model.load_state_dict(torch.load("./4.model.pth"))
model.cuda()
model.eval()

RNNModel(
  (drop): Dropout(p=0.2)
  (encoder): Embedding(85008, 200)
  (rnn): LSTM(200, 200, num_layers=2, dropout=0.2)
  (decoder): Linear(in_features=200, out_features=85008)
)

In [22]:



hidden = model.init_hidden(1)
# input = Variable(torch.rand(1, 1).mul(ntokens).long(), volatile=True)
idx = corpus.dictionary.word2idx['There']
input = Variable(torch.LongTensor([[idx]]).long(), volatile=True)
input.data = input.data.cuda()



print(corpus.dictionary.idx2word[idx], '', end='')

for i in range(num_words):
    output, hidden = model(input, hidden)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    input.data.fill_(word_idx)
    word = corpus.dictionary.idx2word[word_idx]

    #print(word + ('\n' if i % 20 == 19 else ' '), end='')
    if word == '<eos>':
        print('')
    else:
        print(word + ' ', end='')

There was a visit. He was a sure of 
minute or obscure pride, large, of his system. I had to go to 
him come to Mirabeau or investment its decision. Until then had 
drink of hyacinths, hers to the editor and hunting Friday 
Charleses to the beach. 

I took my next frowzy place. I was movin' to tell you, sir, that I was 
about in the poles dollar. 

"Would I had--twenty-four gentian Lowery," Sam," she says. "I never saw 
one run?" "why do this work. 

"Before I could, to whiskey while he turned, and asked High Jack's 
almost in one of the great illness of the lake whom they were 
Dicky's with a wedding banana Hotel, and the druggist had done. I 
have concealed his slippers borrowed with gold hand. Then he possessed 
me his duty into an obscure boxes who gave you cap to a man in front 
of advance, and when the people into island; fainter by 
pile of a petrified eye, with a grin. I always spoke to 
propped around it. But my colour. A black flannel home-hurrying, as 